# Submittin an FL Hub job

In [1]:
# add path
import sys
print(sys.executable)
print(sys.version)

NVFLARE_HOME=f"/home/hroth/Code2/nvflare/flhub"
HUB_EXAMPLE=f"{NVFLARE_HOME}/examples/flhub"
sys.path.append(f"{NVFLARE_HOME}")
sys.path.append(f"{NVFLARE_HOME}/examples")
sys.path.append(f"{NVFLARE_HOME}/integration/monai")

/home/hroth/Code2/nvflare/flhub/examples/flhub/nvflare_flhub/bin/python3.8
3.8.15 (default, Oct 12 2022, 19:15:16) 
[GCC 11.2.0]


In [2]:
# (optional) check environment
!python --version
!pip freeze | grep torch
!pip freeze | grep monai

Python 3.8.15
pytorch-ignite==0.4.10
torch==1.13.0
monai==1.0.1


### Open admin api runner

In [3]:
import msgpack
from nvflare.fuel.hci.client.fl_admin_api_runner import FLAdminAPIRunner, api_command_wrapper

# Initialize the runner
runner = FLAdminAPIRunner(
    username="admin@nvflare.com",
    admin_dir=f"{HUB_EXAMPLE}/workspaces/t1_workspace/admin@nvflare.com",
    poc=False,
    debug=False,
)

### Submit job

In [4]:
job_folder=f"{HUB_EXAMPLE}/job-numpy-sag"
api_command_wrapper(runner.api.submit_job(job_folder))

{'status': <APIStatus.SUCCESS: 'SUCCESS'>, 'details': {'message': 'Submitted job: cb0b5eab-c495-48e7-a53f-ab41512883cf', 'job_id': 'cb0b5eab-c495-48e7-a53f-ab41512883cf'}, 'raw': {'time': '2022-11-07 12:46:51.071888', 'data': [{'type': 'string', 'data': 'Submitted job: cb0b5eab-c495-48e7-a53f-ab41512883cf'}, {'type': 'success', 'data': ''}], 'status': <APIStatus.SUCCESS: 'SUCCESS'>}}


{'status': <APIStatus.SUCCESS: 'SUCCESS'>,
 'details': {'message': 'Submitted job: cb0b5eab-c495-48e7-a53f-ab41512883cf',
  'job_id': 'cb0b5eab-c495-48e7-a53f-ab41512883cf'},
 'raw': {'time': '2022-11-07 12:46:51.071888',
  'data': [{'type': 'string',
    'data': 'Submitted job: cb0b5eab-c495-48e7-a53f-ab41512883cf'},
   {'type': 'success', 'data': ''}],
  'status': <APIStatus.SUCCESS: 'SUCCESS'>}}

### List jobs

In [5]:
%%capture
jobs = api_command_wrapper(runner.api.list_jobs())

In [6]:
print(jobs["raw"]["data"][0]["data"])

----------------------------------------------------------------------------------------------------------------------
| JOB ID                               | NAME          | STATUS  | SUBMIT TIME                      | RUN DURATION   |
----------------------------------------------------------------------------------------------------------------------
| cb0b5eab-c495-48e7-a53f-ab41512883cf | spleen-bundle | RUNNING | 2022-11-07T12:46:51.077819-05:00 | 0:00:01.643938 |
----------------------------------------------------------------------------------------------------------------------



### Show server log

In [11]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("server"))

In [12]:
print(reply["details"]["message"])

2022-11-07 12:47:00,693 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (533ef5db-08bd-4c7f-8b04-5753b70e976f) 
2022-11-07 12:47:01,191 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_b (b035a17a-3e96-4829-a9b8-8858dde53629) 
2022-11-07 12:47:02,829 - FederatedServer - INFO - received update from t1_project_t1_client_a_0 (1047 Bytes, 1667843222 seconds)
2022-11-07 12:47:02,832 - FederatedServer - INFO - received update from t1_project_t1_client_b_0 (1047 Bytes, 1667843222 seconds)
2022-11-07 12:47:04,924 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (533ef5db-08bd-4c7f-8b04-5753b70e976f) 
2022-11-07 12:47:04,931 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_b (b035a17a-3e96-4829-a9b8-8858dde53629) 
2022-11-07 12:47:06,983 - FederatedServer - INFO - received update from t1_project_t1_client_a_0 (1047 Bytes, 1667843226 seconds)
2022-11-07 12:47:07,526 - FederatedServer 

### Show Connector-A log

In [13]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_a"))

In [14]:
print(reply["details"]["message"])

2022-10-31 17:37:48,660 - ClientEngine - INFO - Starting client app. rank: 0
2022-10-31 17:37:48,679 - ProcessExecutor - INFO - Worker child process ID: 138361
2022-10-31 17:37:50,684 - ProcessExecutor - INFO - run (6fada376-ec91-4ca7-a11f-a8c17f93ec05): waiting for child worker process to finish.
2022-10-31 17:38:26,402 - ProcessExecutor - INFO - run (6fada376-ec91-4ca7-a11f-a8c17f93ec05): child worker process finished with execution code: 0
2022-11-07 12:44:26,290 - FederatedClient - INFO - Got the new primary SP: localhost:8002
2022-11-07 12:44:27,348 - FederatedClient - INFO - Successfully registered client:t1_client_a for project t1_project. Token:533ef5db-08bd-4c7f-8b04-5753b70e976f SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
2022-11-07 12:46:52,887 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-07 12:46:52,908 - ProcessExecutor - INFO - Worker child process ID: 1043544
2022-11-07 12:46:54,913 - ProcessExecutor - INFO - run (cb0b5eab-c495-48e7-a53f-ab41512883cf): wait

### Show Connector-B log

In [8]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_b"))

In [9]:
print(reply["details"]["message"])

2022-11-07 12:47:00,693 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (533ef5db-08bd-4c7f-8b04-5753b70e976f) 
2022-11-07 12:47:01,191 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_b (b035a17a-3e96-4829-a9b8-8858dde53629) 
2022-11-07 12:47:02,829 - FederatedServer - INFO - received update from t1_project_t1_client_a_0 (1047 Bytes, 1667843222 seconds)
2022-11-07 12:47:02,832 - FederatedServer - INFO - received update from t1_project_t1_client_b_0 (1047 Bytes, 1667843222 seconds)
2022-11-07 12:47:04,924 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (533ef5db-08bd-4c7f-8b04-5753b70e976f) 
2022-11-07 12:47:04,931 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_b (b035a17a-3e96-4829-a9b8-8858dde53629) 
2022-11-07 12:47:06,983 - FederatedServer - INFO - received update from t1_project_t1_client_a_0 (1047 Bytes, 1667843226 seconds)
2022-11-07 12:47:07,526 - FederatedServer 

### Finish

In [7]:
runner.api.logout()

{'status': <APIStatus.ERROR_INACTIVE_SESSION: 'ERROR_INACTIVE_SESSION'>,
 'details': 'session is logging out'}

# Show TensorBoard

In [8]:
%load_ext tensorboard
%tensorboard --logdir "/home/hroth/Code2/nvflare/flhub"